# Compare different sequential models

Author: Thorsten Kalb

This Notebook should execute the functions given in movielens_sequence and evaluate the validation accuracy.
It will be used later to compare the sequential models to the explicit model.

In [3]:
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
import matplotlib.pyplot as plt
import numpy as np
import sys
import mlflow
from mlflow import log_metric, log_param, log_artifact


dataset_name = '100K'
dataset = get_movielens_dataset(dataset_name)
mlflow.set_tracking_uri('./mlruns')

In [5]:
train, test = user_based_train_test_split(dataset)

train, validation = user_based_train_test_split(train)

train = train.to_sequence()
validation = validation.to_sequence()
test = test.to_sequence()

sequmods = ['pooling','cnn','lstm']

for repres in sequmods:
    with mlflow.start_run():
        n_iter=1
        loss='bpr'
        model = ImplicitSequenceModel(n_iter=n_iter, #10 #5 #3 #50
                                  representation=repres,
                                  loss=loss)
        model.fit(train)
        mrr = sequence_mrr_score(model, validation)
        fig = plt.figure()
        mean_mrr = np.mean(mrr)
        mean_invmrr = np.mean(1/mrr[mrr>0.0])
        # log the model's parameters in mlflow
        log_param("dataset_name",dataset_name)
        log_param("n_iter",n_iter)
        log_param("loss",loss)
        log_metric("mean_mrr",mean_mrr)
        log_metric("mean_invmrr",mean_invmrr)
        log_param("model type",repres)
        
        log_artifact("model_object",model)
        # plot the performance
        plt.plot(mrr,alpha=0.5)
        #plt.yscale('log')
        plt.title(repres)
        plt.xlabel("user")
        plt.ylabel("mrr_score")
        plt.axhline(y=mean_mrr, color='r', linestyle='-')
        print("Mean mrr validation score: {:.4f} for model {}".format(mean_mrr,repres))
        print("On average, the first relevant movie int the predicted sequence is at place {} for model {}".format(mean_invmrr,repres))
    #mlflow.end_run()


TypeError: expected str, bytes or os.PathLike object, not ImplicitSequenceModel

<Figure size 432x288 with 0 Axes>

In [3]:
#!mlflow ui --backend-store-uri ./mlruns

## Result:

It depends a lot on the **number of epochs** during training, which model performs better. However, for few iterations, the CNN and lstm have an average mrr score of 0.2m while the pooling model has an average mrr score of 0.4. For more epochs, the validation mrr score increases for all models. 

For 50 epochs, I report a mrr scores of 0.058, 0.056 and 0.023 for the pooling, cnn and lstm models, respectively. 
For 100 epochs, I report mrr scores of 0.055, 0.043 and 0.045 for the pooling, cnn and lstm models, respectively.
For a better comparison, these experiments should be saved in a docker image with fixed seed etc. 

I propose **new metrics** for the evaluation here, namely, when does the **first relevant movie** appear in the list, on average. Therefore, I take the average over users of all the inverses of the mrr_scores, wherever the inverse exists.

Thus, I would **choose the Pooling model**. The advantage is a higher starting mrr score and a consistently higher mrr score, which means we do not need to train the model a lot and thus, it is quickly accesible and highly adaptable to new data.

A **direct comparison with the Explicit model is not (yet) possible**. This is, because of the different nature of the models: Sequential models try to predict the next element of a sequence, whereas the explicit model tries to predict the rating of the user on a movie that the user has not yet seen. Perhaps in the future, we can work on a way to compare the models, e.g. by ordering the rankings from the explicit model and turning this into a sequence?

Interestingly, the **mrr scores change for bigger datasets**, resulting in CNN outperforming the pooling, for the given 50 training epochs and evaluating the mean of the mrr score. Interestingly, the mean place, with the first relevant movie is still better for the pooling model for the 1M dataset. The LSTM model worsens in both aspects significantly for the 1M dataset.
